In [12]:
import numpy as np
import pandas as pd
import sympy as sp
from IPython.display import display, clear_output

In [13]:
class Laguerre:
    def __init__(self, T, N, beta, sigma, epsilon, t_values):
        self.T = T
        self.N = N
        self.beta = beta
        self.sigma = sigma
        self.epsilon = epsilon
        self.t_values = t_values
        self.alpha = self.sigma - self.beta
    
    
    @property
    def T(self):
        return self._T
    @T.setter
    def T(self, T):
        self._T = T
        
    @property
    def N(self):
        return self._N
    @N.setter
    def N(self, N):
        self._N = N
               
    @property
    def beta(self):
        return self._beta
    @beta.setter
    def beta(self, beta):
        self._beta = beta
        
    @property
    def sigma(self):
        return self._sigma
    @sigma.setter
    def sigma(self, sigma):
        self._sigma = sigma        
        
    @property
    def epsilon(self):
        return self._epsilon
    @epsilon.setter
    def epsilon(self, epsilon):
        self._epsilon = epsilon
          
    @property
    def t_values(self):
        return self._t_values
    @t_values.setter
    def t_values(self, t_values):
        self._t_values = t_values
        
    def laguerre(self, n, t):
        if (self.beta>=0) and (self.beta<=self.sigma):
            if n==0:
                return np.sqrt(self.sigma)*np.exp((-self.beta*t)/2)
            elif n==1:
                return np.sqrt(self.sigma)*(1-self.sigma*t)*np.exp((-self.beta*t)/2)
            else:
                l0=np.sqrt(self.sigma)*np.exp((-self.beta*t)/2)
                l1=np.sqrt(self.sigma)*(1-self.sigma*t)*np.exp((-self.beta*t)/2)
                for i in range(2, n+1):
                    li=((2*i-1-self.sigma*t)/i)*l1 - ((i-1)/i)*l0
                    l0, l1 = l1, li
                return li

        else:
            raise ValueError("Wrong data input! Beta should be in range [0;sigma]!")
        
    
    def tabulate_laguerre(self): 
        laguerre_val = {"t_val": self.t_values}  
        for n in range(self.N + 1):
            laguerre_val[f"L_{n}"] = [self.laguerre(n, t) for t in self.t_values]
        
        return pd.DataFrame(laguerre_val)
    
        
    def find_t_epsilon(self, laguerre_values):
        for index, row in laguerre_values.iterrows():
            lag_val=row.drop("t_val")
            if (lag_val.abs() < self.epsilon).all():
                return row["t_val"]
        return None

In [14]:
def parse_function(f_str):
    """Парсер стрінга у функу"""
    return sp.sympify(f_str)


In [15]:

class Integral(Laguerre):
    def __init__(self, T, N, beta, sigma, epsilon, t_values, max_steps=10000):
        super().__init__(T, N, beta, sigma, epsilon, t_values)
        self.max_steps = max_steps

    def rect_integrate_laguerre(self, n, f):
        """Інтегрування методом прямокутників"""
        steps = 1
        prev_integral = 0
        error = float("inf")
        
        while error > self.epsilon:
            t_vals = np.linspace(0, self.T, steps + 1)
            delta_t = self.T / steps
            
            laguerre_values = np.array([self.laguerre(n, t) for t in t_vals])
            func_to_integr = f(t_vals) * laguerre_values * np.exp(-self.alpha * t_vals)
           
            integral = np.sum(func_to_integr * delta_t)
            
            error = abs(integral - prev_integral)
            prev_integral = integral
            
            steps *= 2
            if steps > self.max_steps:
                raise RuntimeError("Максимальна кількість кроків перевищена. Інтеграл не збігається.")
        
        return prev_integral
    
    def trapez_integrate_laguerre(self, n, f):
        """Інтегрування методом трапецій"""
        steps = 1
        prev_integral = 0
        error = float("inf")
        
        while error > self.epsilon:
            t_vals = np.linspace(0, self.T, steps + 1)
            delta_t = self.T / steps
            
            laguerre_values = np.array([self.laguerre(n, t) for t in t_vals])
            func_to_integr = f(t_vals) * laguerre_values * np.exp(-self.alpha * t_vals)
            
            integral = delta_t * (0.5 * func_to_integr[0] + 0.5 * func_to_integr[-1] + np.sum(func_to_integr[1:-1]))
            
            error = abs(integral - prev_integral)
            prev_integral = integral
            
            steps *= 2
            if steps > self.max_steps:
                raise RuntimeError("Максимальна кількість кроків перевищена. Інтеграл не збігається.")
        
        return prev_integral

In [ ]:
class ReverseLaguerre(Laguerre):
    def __init__(self, coefficients, T, N, beta, sigma, epsilon, t_values):
        super().__init__(T=T, N=N, beta=beta, sigma=sigma, epsilon=epsilon, t_values=t_values)
        self.coefficients = coefficients

    @property
    def coefficients(self):
        return self._coefficients

    @coefficients.setter
    def coefficients(self, coefficients):
        self._coefficients = coefficients

    def reverse_laguerre(self, integr_obj, t):
        result = 0
        for k in range(len(self.coefficients)):
            result += self.coefficients[k] * integr_obj.rect_integrate_laguerre(k, t)
        return result
    
    def laguerre_coefficients(self,integr_obj, f):
        coeffs = []
        for k in range(len(self.coefficients)):
            coeff =  integr_obj.rect_integrate_laguerre(k, f)
            coeffs.append(coeff)
        return coeffs

    def conditional_func(self, laguerre_obj, t_values=None):
        if t_values is None:
            t_values = np.linspace(0, self.T, 100)
        return np.array([self.reverse_laguerre(laguerre_obj, t) for t in t_values])